In [1]:
# import the packages we'll need
import pandas as pd
import pybaseball as pb

In [17]:
# pybaseball has pitchers and batters split up into different functions
# we'll bring that data in and just grab each players yearly WAR totals
# we end up adding names later. For now we'll just grab the player_ID
dfb = pb.bwar_bat()
dfp = pb.bwar_pitch()

df = pd.concat([dfb[["player_ID", "WAR"]], dfp[["player_ID", "WAR"]]], axis=0)

In [18]:
# now we group all the yearly WAR into career totals
career_war_df = df.groupby("player_ID").WAR.sum().reset_index()
career_war_df["WAR"] = career_war_df.WAR.round(1)

In [19]:
# here we bring in the biographical data, most importantly the first and last names, from pb.people() and join to our carrer WAR
df_name = career_war_df.merge(
    pb.people(),
    left_on = "player_ID",
    right_on = "playerID"
)

In [20]:
# split out the initials into their own columns

df_name["firstInitial"] = df_name.nameFirst.str[0]
df_name["lastInitial"] = df_name.nameLast.str[0]

In [23]:
# filter the dataset down to only players that have the same first and last initial
same_initials_df = df_name[df_name.firstInitial == df_name.lastInitial]

In [28]:
# finally, we group each initial pair and aggregate by size(count), sum, mean and median
result_df = same_initials_df\
    .groupby(["firstInitial", "lastInitial"])\
    .WAR\
    .agg(["size","sum","mean", "median"])\
    .reset_index()
result_df

,firstInitial,lastInitial,size,sum,mean,median
0,A,A,41,84.3,2.056098,0.20
1,B,B,235,1085.1,4.617447,0.10
2,C,C,124,526.5,4.245968,0.05
3,D,D,77,541.0,7.025974,0.60
4,E,E,12,50.2,4.183333,0.20
5,F,F,24,247.7,10.320833,0.15
6,G,G,63,310.6,4.930159,0.20
7,H,H,53,234.1,4.416981,0.10
8,J,J,72,273.4,3.797222,0.00
9,K,K,27,131.1,4.855556,-0.10


In [31]:
# present in markdown for easy sharing
print(result_df.to_markdown(index=False))

| firstInitial   | lastInitial   |   size |    sum |      mean |   median |
|:---------------|:--------------|-------:|-------:|----------:|---------:|
| A              | A             |     41 |   84.3 |  2.0561   |     0.2  |
| B              | B             |    235 | 1085.1 |  4.61745  |     0.1  |
| C              | C             |    124 |  526.5 |  4.24597  |     0.05 |
| D              | D             |     77 |  541   |  7.02597  |     0.6  |
| E              | E             |     12 |   50.2 |  4.18333  |     0.2  |
| F              | F             |     24 |  247.7 | 10.3208   |     0.15 |
| G              | G             |     63 |  310.6 |  4.93016  |     0.2  |
| H              | H             |     53 |  234.1 |  4.41698  |     0.1  |
| J              | J             |     72 |  273.4 |  3.79722  |     0    |
| K              | K             |     27 |  131.1 |  4.85556  |    -0.1  |
| L              | L             |     33 |  105.7 |  3.20303  |    -0.1  |
| M         

In [ ]:
# beyond this point there are a few additional lookups and whatnot for specific research

In [56]:
df_name[
    (df_name.firstInitial == 'K')
    & (df_name.lastInitial == 'K')
][["nameFirst", "nameLast", "WAR"]]

,nameFirst,nameLast,WAR
9222,Kila,Ka'aihue,-0.3
9224,Kevin,Kaczmarski,-0.1
9300,Kenshin,Kawakami,1.0
9346,Katsy,Keifer,0.3
9350,Keone,Kela,5.0
9366,Kris,Keller,-0.1
9367,Kyle,Keller,-0.4
9403,Kenny,Kelly,-0.1
9404,King,Kelly,45.9
9405,Kick,Kelly,-0.8


In [44]:
def lookup_group_sum(row):
    firstInitial = row.firstInitial
    lastInitial = row.lastInitial
    group_WAR_total = result_df[(result_df.firstInitial == firstInitial)
            & (result_df.lastInitial == lastInitial)
    ].iloc[0]["sum"]
    return group_WAR_total

In [48]:
same_initials_df["groupWAR"] = same_initials_df.apply(lookup_group_sum, axis=1)

/home/jup/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [51]:
same_initials_df["pct_of_total"] = same_initials_df.WAR / same_initials_df.groupWAR.abs()

/home/jup/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [54]:
same_initials_df.sort_values("pct_of_total", ascending=False)[["nameFirst", "nameLast", "WAR", "groupWAR", "pct_of_total"]].head(20)

,nameFirst,nameLast,WAR,groupWAR,pct_of_total
18508,Vince,Velasquez,4.5,4.2,1.071429
19860,Zip,Zabel,2.2,2.2,1.000000
18342,Ugueth,Urbina,13.2,13.2,1.000000
5227,Edwin,Encarnacion,35.4,50.2,0.705179
14297,Placido,Polanco,41.9,67.9,0.617084
13541,Orval,Overall,22.4,38.6,0.580311
19757,Yam,Yaryan,0.2,-0.4,0.500000
13382,Omar,Olivares,15.5,38.6,0.401554
9404,King,Kelly,45.9,131.1,0.350114
18406,Vito,Valentinetti,1.4,4.2,0.333333
